## 1. 환경 설정 및 라이브러리 임포트

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError('OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.')

print(f'✅ API 키 설정 완료')

# 실시간 현재 시간 확인
from datetime import datetime
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f'✅ 현재 시간: {current_time}')


✅ API 키 설정 완료
✅ 현재 시간: 2025-12-03 15:21:43


In [2]:
# LangChain 관련 임포트
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print('✅ LangChain 라이브러리 임포트 완료')

✅ LangChain 라이브러리 임포트 완료


In [3]:
# 날짜 필터링 유틸리티 함수
from datetime import datetime
import re

def parse_date_range(date_str: str) -> tuple:
    """
    신청기간 문자열을 파싱하여 (시작일, 종료일) 튜플 반환
    예: '20251125 ~ 20251204' -> (datetime(2025,11,25), datetime(2025,12,4))
    
    Returns:
        tuple: (start_date, end_date) or (None, None) if parsing fails
    """
    if not date_str or date_str == 'N/A' or date_str.strip() == '':
        return (None, None)
    
    try:
        # '~' 또는 '-'로 분리
        parts = re.split(r'\s*[~\-]\s*', date_str.strip())
        
        if len(parts) == 2:
            start_str, end_str = parts
            # YYYYMMDD 형식 파싱
            start_date = datetime.strptime(start_str.strip(), '%Y%m%d')
            end_date = datetime.strptime(end_str.strip(), '%Y%m%d')
            return (start_date, end_date)
        elif len(parts) == 1:
            # 단일 날짜인 경우
            single_date = datetime.strptime(parts[0].strip(), '%Y%m%d')
            return (single_date, single_date)
    except (ValueError, AttributeError):
        pass
    
    return (None, None)


def is_currently_available(date_str: str, today: datetime = None) -> bool:
    """
    현재 신청 가능한지 확인 (1단계)
    - 빈 문자열/N/A는 상시 모집으로 간주 → True
    - 오늘 날짜가 신청기간 내에 있으면 True
    """
    if today is None:
        today = datetime.now()
    
    # 빈 값은 상시 모집으로 간주
    if not date_str or date_str == 'N/A' or date_str.strip() == '':
        return True
    
    start_date, end_date = parse_date_range(date_str)
    
    # 파싱 실패시 상시 모집으로 간주
    if start_date is None or end_date is None:
        return True
    
    # 오늘이 신청기간 내인지 확인
    return start_date <= today <= end_date


def is_this_year(date_str: str, year: int = None) -> bool:
    """
    올해 진행 중인 정책인지 확인 (2단계)
    - 빈 문자열/N/A는 상시 모집으로 간주 → True
    - 신청기간이 올해에 걸쳐있으면 True
    """
    if year is None:
        year = datetime.now().year
    
    # 빈 값은 상시 모집으로 간주
    if not date_str or date_str == 'N/A' or date_str.strip() == '':
        return True
    
    start_date, end_date = parse_date_range(date_str)
    
    # 파싱 실패시 포함
    if start_date is None or end_date is None:
        return True
    
    # 신청기간이 올해에 걸쳐있는지 확인
    year_start = datetime(year, 1, 1)
    year_end = datetime(year, 12, 31)
    
    # 기간이 올해와 겹치는지 확인
    return not (end_date < year_start or start_date > year_end)


def filter_docs_by_time(docs: list, time_mode: int, today: datetime = None) -> list:
    """
    시간 기반으로 문서 필터링
    
    Args:
        docs: 검색된 문서 리스트
        time_mode: 1(현재 신청 가능), 2(올해), 3(전체)
        today: 기준 날짜 (기본: 현재)
    
    Returns:
        list: 필터링된 문서 리스트
    """
    if today is None:
        today = datetime.now()
    
    if time_mode == 3:  # 전체 - 필터링 없음
        return docs
    
    filtered = []
    for doc in docs:
        date_str = doc.metadata.get('신청기간', '')
        
        if time_mode == 1:  # 현재 신청 가능
            if is_currently_available(date_str, today):
                filtered.append(doc)
        elif time_mode == 2:  # 올해 진행
            if is_this_year(date_str, today.year):
                filtered.append(doc)
    
    return filtered


print('✅ 날짜 필터링 유틸리티 함수 정의 완료')
print(f'   - parse_date_range(): 날짜 문자열 파싱')
print(f'   - is_currently_available(): 현재 신청 가능 여부')
print(f'   - is_this_year(): 올해 진행 여부')
print(f'   - filter_docs_by_time(): 시간 기반 문서 필터링')

✅ 날짜 필터링 유틸리티 함수 정의 완료
   - parse_date_range(): 날짜 문자열 파싱
   - is_currently_available(): 현재 신청 가능 여부
   - is_this_year(): 올해 진행 여부
   - filter_docs_by_time(): 시간 기반 문서 필터링


In [4]:
# LLM 기반 의도 분류기 (시간 범위 판단)
INTENT_CLASSIFIER_PROMPT = '''당신은 청년 정책 질문의 의도를 분류하는 분류기입니다.
사용자의 질문을 분석하여 어떤 시간 범위의 정책을 원하는지 판단하세요.

## 분류 기준
1: 현재 신청 가능한 정책 (기본값)
   - 일반적인 정책 질문 ("취업 지원 뭐 있어?", "주거 정책 알려줘")
   - "지금", "현재", "신청 가능한" 등의 키워드
   
2: 올해 진행 중인 정책
   - "올해", "이번 년도", "2025년", "금년" 등의 키워드
   - "올해 뭐 있었어?", "이번 년도 정책" 등
   
3: 시간 무관 (역대 전체)
   - "전체", "모든", "역대", "지금까지", "과거", "종료된 것도"
   - "다 알려줘", "전부", "예전 정책도"

## 규칙
- 명확한 시간 키워드가 없으면 1(현재 신청 가능)로 분류
- 숫자만 응답하세요 (1, 2, 또는 3)

질문: {question}

분류 (1/2/3):'''


def classify_time_intent(question: str, llm) -> int:
    """
    LLM을 사용하여 질문의 시간 범위 의도 분류
    
    Args:
        question: 사용자 질문
        llm: ChatOpenAI 인스턴스
    
    Returns:
        int: 1(현재), 2(올해), 3(전체)
    """
    prompt = ChatPromptTemplate.from_template(INTENT_CLASSIFIER_PROMPT)
    chain = prompt | llm | StrOutputParser()
    
    try:
        result = chain.invoke({'question': question})
        # 숫자만 추출
        time_mode = int(result.strip()[0])
        if time_mode in [1, 2, 3]:
            return time_mode
    except (ValueError, IndexError):
        pass
    
    # 기본값: 현재 신청 가능
    return 1


print('✅ LLM 기반 의도 분류기 정의 완료')
print('   - classify_time_intent(): 질문에서 시간 범위 의도 추출')

✅ LLM 기반 의도 분류기 정의 완료
   - classify_time_intent(): 질문에서 시간 범위 의도 추출


## 2. VectorDB 로드

기존에 구축된 ChromaDB(youth_policies 컬렉션)를 로드합니다.

In [5]:
# 프로젝트 경로 설정
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir) if current_dir.endswith('src') else current_dir
db_path = os.path.join(project_root, 'data', 'vectordb')

print(f'📁 VectorDB 경로: {db_path}')
print(f'📁 경로 존재 여부: {os.path.exists(db_path)}')

📁 VectorDB 경로: c:\SKN20-3rd-1TEAM\SKN20-3rd-1TEAM\data\vectordb
📁 경로 존재 여부: True


In [6]:
# 임베딩 모델 설정
# 모델 테스트 단계라 추후 정리 할 예정입니다.

# # 1. OpenAI 모델
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
print('✅ OpenAI API 임베딩 모델 선택 완료')


# # 2.KoSimCSE-RoBERTa) (고성능)
# from langchain_community.embeddings import HuggingFaceEmbeddings
# model_name = "BM-K/KoSimCSE-roberta-large" 
# embedding_model = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cpu'}, # GPU가 없다면 'cpu' 사용
#     encode_kwargs={'normalize_embeddings': True}
# )
# print(f'✅ 로컬 임베딩 모델 선택 완료: {model_name}')


# # 3. KCBERT-Base (범용 안정성)
# from langchain_community.embeddings import HuggingFaceEmbeddings
# model_name = "beomi/kcbert-base"
# embedding_model = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cpu'}, 
#     encode_kwargs={'normalize_embeddings': True}
# )
# print(f'✅ 로컬 임베딩 모델 선택 완료: {model_name}')


# # 4. SKT KoBERT-SBERT (빠른 속도)
# from langchain_community.embeddings import HuggingFaceEmbeddings
# model_name = "skt/kobert-base-v1" 
# embedding_model = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cpu'}, 
#     encode_kwargs={'normalize_embeddings': True}
# )
# print(f'✅ 로컬 임베딩 모델 선택 완료: {model_name}')
# 기존 ChromaDB 로드
vectorstore = Chroma(
    persist_directory=db_path,
    collection_name='youth_policies',
    embedding_function=embedding_model
)

policy_count = vectorstore._collection.count()
print(f'✅ VectorDB 로드 완료')
print(f'📊 저장된 정책 수: {policy_count}개')

✅ OpenAI API 임베딩 모델 선택 완료
✅ VectorDB 로드 완료
📊 저장된 정책 수: 3000개


## 3. RAG 체인 구성

리트리버, 프롬프트, LLM을 연결하여 RAG 체인을 구성합니다.

In [7]:
# 리트리버 설정
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 5}
)

print('✅ 리트리버 설정 완료 (top_k=5)')

✅ 리트리버 설정 완료 (top_k=5)


In [8]:
# 문서 포맷팅 함수
def format_docs(docs):
    """검색된 문서를 문자열로 포맷팅"""
    formatted = []
    for i, doc in enumerate(docs, 1):
        metadata = doc.metadata
        text = f"""
[정책 {i}]
정책명: {metadata.get('정책명', 'N/A')}
분야: {metadata.get('대분류', '')} > {metadata.get('중분류', '')}
주관기관: {metadata.get('주관기관명', 'N/A')}
신청기간: {metadata.get('신청기간', 'N/A')}
신청방법: {metadata.get('신청방법', 'N/A')}
지원연령: {metadata.get('지원최소연령', '0')}세 ~ {metadata.get('지원최대연령', '0')}세
지원금액: {metadata.get('최소지원금액', '0')}원 ~ {metadata.get('최대지원금액', '0')}원
신청URL: {metadata.get('신청URL', 'N/A')}

내용:
{doc.page_content}
"""
        formatted.append(text)
    return '\n\n---\n\n'.join(formatted)

print('✅ 문서 포맷팅 함수 정의 완료')

✅ 문서 포맷팅 함수 정의 완료


In [9]:
# LLM 설정
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

print('✅ LLM 설정 완료 (gpt-4o-mini)')

✅ LLM 설정 완료 (gpt-4o-mini)


In [10]:
# 프롬프트 템플릿 정의 (전역 변수로 관리 - 클래스에서도 재사용)
SYSTEM_PROMPT = '''당신은 "청년이음"의 스마트한 선배입니다.
복잡한 청년 정책을 후배에게 쉽고 친근하게 설명해주는 역할입니다.

## 현재 시간 정보
- 오늘 날짜: {today}

## 페르소나
- 이름: 스마트한 선배 (Smart Mentor)
- 성격: 친절하고 격려하며, 명확하게 설명함
- 말투: 대학 선배가 후배에게 알려주듯 편안하고 이해하기 쉽게 (반말 사용)

## 답변 형식 (반드시 이 구조를 따르세요)

### 1️⃣ 인사/공감 (첫 1~2문장)
밝고 긍정적인 분위기로 시작하세요.
예시:
- "요즘 이 정책 궁금해하는 사람 많더라구!"
- "오, 좋은 질문이야! 도움 될 만한 정보가 있어서 공유하고 싶었어!"
- "이거 진짜 알아두면 좋은 정책이야!"

### 2️⃣ 정책 핵심 요약
"{{정책명}}은(는) {{정책 목적/핵심 요약}}에 도움이 되는 정책이야." 형식으로 설명하세요.

### 3️⃣ 구조화된 안내 (이모지 포함)
아래 형식을 사용하세요:
✨ 지원 내용: {{지원내용}} 
📝 신청 자격: {{자격조건}} 
🗓️ 신청 기간: {{신청기간 - 제공된 원본 데이터 그대로 표시}} 
🧾 신청 방법: {{신청방법}} 
🔗 신청하러 가기: {{URL}} (있을 경우만)

### 4️⃣ 마무리 (밝고 권유형)
예시:
- "조건 맞으면 꼭 한 번 확인해보는 거 추천해!"
- "이건 진짜 놓치기 아쉬운 정책이야."
- "해당되면 바로 신청해봐!"

### 5️⃣ 추가 안내
마지막에 항상 이 문장을 포함하세요:
"더 궁금한 거 있으면 편하게 물어봐! 😊"

## 답변 규칙
1. 제공된 정책 정보만을 기반으로 답변하세요.
2. 어려운 행정 용어는 쉽게 풀어서 설명하세요.
3. 여러 정책이 있으면 각각 위 형식으로 안내하세요.
4. 해당하는 정책이 없으면: "음, 아쉽지만 딱 맞는 정책을 찾지 못했어. 다른 키워드로 다시 물어봐줄래? 😊"
5. 반말을 사용하되 존중하는 톤을 유지하세요.
6. "총 몇 개", "전체 개수" 등 총 정책 수를 묻는 질문에는 반드시 제공된 [총 정책 수]를 사용해서 답변하세요. 검색된 정책 개수를 총 개수라고 착각하지 마세요.
7. 사용자가 특정 개수(예: 6개, 10개)를 요청하면 제공된 정책 중에서 해당 개수만큼만 안내하세요. 제공된 정책이 부족하면 있는 만큼만 안내하고 "더 많은 정책이 있을 수 있어! 구체적인 조건을 알려주면 더 찾아볼게."라고 덧붙이세요.
8. 신청기간은 제공된 원본 데이터를 그대로 표시하세요. (예: "20251125 ~ 20251204", "N/A" 등)
9. 신청기간이 "N/A"이거나 비어있으면 "상시 모집 또는 별도 공지"라고 안내하세요.

## 정책 분야 참고
- 🏢 일자리: 취업 장려금, 창업 지원, 직무 교육, 인턴십
- 🏠 주거: 전월세 보증금 대출, 월세 지원, 공공임대
- 🎓 교육: 장학금, 학자금 대출, 자격증 응시료 지원
- 🎨 복지/문화: 청년 수당, 마음 건강 지원, 문화 바우처
- ⚖️ 참여/권리: 정책 제안, 청년 공간, 법률 상담
'''

HUMAN_PROMPT = '''
[검색된 관련 정책 정보]:
{context}

질문: {question}

답변:'''


# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_messages([
    ('system', SYSTEM_PROMPT),
    ('human', HUMAN_PROMPT)
])

print('✅ 프롬프트 템플릿 정의 완료')

✅ 프롬프트 템플릿 정의 완료


In [11]:
# LCEL 방식 RAG 체인 구성
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print('✅ RAG 체인 구성 완료')

✅ RAG 체인 구성 완료


## 4. RAG 클래스 정의

사용하기 편리하도록 RAG 시스템을 클래스로 래핑합니다.

In [12]:
class YouthPolicyRAG:
    """청년 정책 RAG 시스템 클래스 (시간 기반 필터링 내장)"""
    
    def __init__(self, db_path, retriever_k=5, search_k_multiplier=4, default_time_mode=1):
        """
        RAG 시스템 초기화
        
        Args:
            db_path: VectorDB 경로
            retriever_k: 최종 반환할 문서 개수
            search_k_multiplier: 필터링 전 검색 배수 (retriever_k * multiplier 만큼 검색)
            default_time_mode: 기본 시간 모드 (1: 현재 신청 가능)
        """
        print('=' * 60)
        print('🚀 청년 정책 RAG 시스템 초기화')
        print('   💡 기본 설정: 현재 신청 가능한 정책만 검색')
        print('=' * 60)
        
        # 임베딩 모델
        self.embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
        
        # VectorDB 로드
        self.vectorstore = Chroma(
            persist_directory=db_path,
            collection_name='youth_policies',
            embedding_function=self.embedding_model
        )
        self.total_policies = self.vectorstore._collection.count()
        print(f'✅ VectorDB 로드 완료 (정책 수: {self.total_policies}개)')
        
        # 검색 설정
        self.retriever_k = retriever_k
        self.search_k = retriever_k * search_k_multiplier
        self.default_time_mode = default_time_mode
        
        # LLM 설정
        self.llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
        
        print(f'✅ 검색 설정: 선 검색 {self.search_k}개 → 시간 필터링 → {self.retriever_k}개 반환')
        print('✅ RAG 시스템 준비 완료!\n')
    
    def _classify_intent(self, question: str) -> int:
        """LLM을 사용하여 시간 범위 의도 분류"""
        return classify_time_intent(question, self.llm)
    
    def _search_and_filter(self, query: str, time_mode: int, top_k: int = None) -> list:
        """
        검색 후 시간 기반 필터링 수행 (핵심 로직)
        
        Args:
            query: 검색 쿼리
            time_mode: 1(현재), 2(올해), 3(전체)
            top_k: 반환할 문서 수 (기본: self.retriever_k)
        
        Returns:
            list: 필터링된 문서 리스트
        """
        if top_k is None:
            top_k = self.retriever_k
        
        # 더 많은 문서를 먼저 검색
        docs = self.vectorstore.similarity_search(query, k=self.search_k)
        
        # 시간 기반 필터링 (기본 동작)
        filtered_docs = filter_docs_by_time(docs, time_mode)
        
        # top_k 개수만큼 반환
        return filtered_docs[:top_k]
    
    def _format_docs(self, docs: list) -> str:
        """검색된 문서를 포맷팅 (원본 데이터 그대로)"""
        formatted = []
        
        for i, doc in enumerate(docs, 1):
            metadata = doc.metadata
            
            text = f"""
[정책 {i}]
정책명: {metadata.get('정책명', 'N/A')}
분야: {metadata.get('대분류', '')} > {metadata.get('중분류', '')}
주관기관: {metadata.get('주관기관명', 'N/A')}
신청기간: {metadata.get('신청기간', 'N/A')}
신청방법: {metadata.get('신청방법', 'N/A')}
지원연령: {metadata.get('지원최소연령', '0')}세 ~ {metadata.get('지원최대연령', '0')}세
지원금액: {metadata.get('최소지원금액', '0')}원 ~ {metadata.get('최대지원금액', '0')}원
신청URL: {metadata.get('신청URL', 'N/A')}

내용:
{doc.page_content}
"""
            formatted.append(text)
        return '\n\n---\n\n'.join(formatted)
    
    def ask(self, question: str, time_mode: int = None) -> str:
        """
        질문에 대한 답변 생성
        
        Args:
            question: 사용자 질문
            time_mode: 시간 모드 (None이면 LLM이 자동 판단)
        """
        # 시간 모드 결정 (기본값: 자동 판단)
        if time_mode is None:
            time_mode = self._classify_intent(question)
        
        # 검색 및 필터링 (시간 필터링은 내부적으로 자동 적용)
        docs = self._search_and_filter(question, time_mode)
        
        # 컨텍스트 생성 (원본 데이터 그대로)
        context = self._format_docs(docs)
        
        # 프롬프트 생성
        today_str = datetime.now().strftime('%Y-%m-%d')
        
        prompt = ChatPromptTemplate.from_messages([
            ('system', SYSTEM_PROMPT),
            ('human', HUMAN_PROMPT)
        ])
        
        chain = prompt | self.llm | StrOutputParser()
        
        return chain.invoke({
            'context': context,
            'question': question,
            'today': today_str
        })
    
    def ask_with_sources(self, question: str, time_mode: int = None) -> dict:
        """
        질문에 대한 답변과 출처 반환
        
        Args:
            question: 사용자 질문
            time_mode: 시간 모드 (None이면 LLM이 자동 판단)
        """
        # 시간 모드 결정
        if time_mode is None:
            time_mode = self._classify_intent(question)
        
        # 검색 및 필터링
        docs = self._search_and_filter(question, time_mode)
        
        # 컨텍스트 생성
        context = self._format_docs(docs)
        
        # 프롬프트 생성
        today_str = datetime.now().strftime('%Y-%m-%d')
        
        prompt = ChatPromptTemplate.from_messages([
            ('system', SYSTEM_PROMPT),
            ('human', HUMAN_PROMPT)
        ])
        
        chain = prompt | self.llm | StrOutputParser()
        
        answer = chain.invoke({
            'context': context,
            'question': question,
            'today': today_str
        })
        
        # 출처 정보 추출 (신청기간 원본 그대로)
        sources = []
        for doc in docs:
            sources.append({
                '정책명': doc.metadata.get('정책명', 'N/A'),
                '분야': doc.metadata.get('중분류', 'N/A'),
                '주관기관': doc.metadata.get('주관기관명', 'N/A'),
                '신청URL': doc.metadata.get('신청URL', 'N/A'),
                '신청기간': doc.metadata.get('신청기간', 'N/A')
            })
        
        return {
            'answer': answer,
            'sources': sources,
            'retrieved_count': len(docs)
        }
    
    def search_policies(self, query: str, top_k: int = 5, time_mode: int = None) -> list:
        """
        정책 검색만 수행 (LLM 호출 없이)
        
        Args:
            query: 검색 쿼리
            top_k: 반환할 문서 수
            time_mode: 시간 모드 (None이면 기본값 사용)
        """
        if time_mode is None:
            time_mode = self.default_time_mode
            
        docs = self._search_and_filter(query, time_mode, top_k)
        
        results = []
        for doc in docs:
            results.append({
                '정책명': doc.metadata.get('정책명', 'N/A'),
                '분야': f"{doc.metadata.get('대분류', '')} > {doc.metadata.get('중분류', '')}",
                '주관기관': doc.metadata.get('주관기관명', 'N/A'),
                '신청기간': doc.metadata.get('신청기간', 'N/A'),
                '지원연령': f"{doc.metadata.get('지원최소연령', '0')}세 ~ {doc.metadata.get('지원최대연령', '0')}세",
                '신청URL': doc.metadata.get('신청URL', 'N/A'),
                '내용': doc.page_content[:200] + '...'
            })
        
        return results

print('✅ YouthPolicyRAG 클래스 정의 완료')
print('   - 시간 필터링이 기본 동작으로 내장됨')
print('   - 기본값: 현재 신청 가능한 정책만 검색')
print('   - 질문에 따라 LLM이 자동으로 시간 범위 판단')

✅ YouthPolicyRAG 클래스 정의 완료
   - 시간 필터링이 기본 동작으로 내장됨
   - 기본값: 현재 신청 가능한 정책만 검색
   - 질문에 따라 LLM이 자동으로 시간 범위 판단


## 5. 테스트 실행

RAG 시스템을 테스트합니다.

In [13]:
# RAG 시스템 초기화
rag = YouthPolicyRAG(db_path=db_path, retriever_k=5)

🚀 청년 정책 RAG 시스템 초기화
   💡 기본 설정: 현재 신청 가능한 정책만 검색
✅ VectorDB 로드 완료 (정책 수: 3000개)
✅ 검색 설정: 선 검색 20개 → 시간 필터링 → 5개 반환
✅ RAG 시스템 준비 완료!



In [14]:
# 간단한 검색 테스트 (LLM 호출 없이)
search_results = rag.search_policies('취업 지원', top_k=3)

print('🔍 검색 결과 (취업 지원):\n')
for i, policy in enumerate(search_results, 1):
    print(f"[{i}] {policy['정책명']}")
    print(f"    분야: {policy['분야']}")
    print(f"    주관기관: {policy['주관기관']}")
    print()

🔍 검색 결과 (취업 지원):

[1] 국민취업지원제도
    분야: 일자리 > 취업
    주관기관: 고용노동부

[2] 청년취업지원센터(도봉구)
    분야: 일자리,일자리 > 취업,재직자
    주관기관: 도봉구 청년미래과

[3] 국민취업지원제도
    분야: 일자리 > 취업
    주관기관: 고용노동부



In [15]:
# 질문-답변 테스트 1
question = '취업 지원 정책이 있나요?'

print(f'👤 질문: {question}')
print('─' * 60)

result = rag.ask_with_sources(question)

print(f'\n🤖 답변:\n{result["answer"]}')
print(f'\n📚 참고 정책 ({result["retrieved_count"]}개):')
for i, source in enumerate(result['sources'], 1):
    print(f"  {i}. {source['정책명']} ({source['분야']})")

👤 질문: 취업 지원 정책이 있나요?
────────────────────────────────────────────────────────────

🤖 답변:
오, 좋은 질문이야! 요즘 취업 지원 정책이 많이 나와서 도움이 될 거야.

### 1️⃣ 국민취업지원제도
국민취업지원제도는 청년실업자 등에게 원하는 일자리를 찾을 수 있도록 취업지원서비스와 소득지원을 함께 제공하여 노동시장 진입촉진과 경제적 자립기반 마련에 도움이 되는 정책이야.

✨ 지원 내용: 맞춤형 취업지원서비스와 소득지원 제공  
📝 신청 자격: 15세 ~ 34세  
🗓️ 신청 기간: 상시 모집 또는 별도 공지  
🧾 신청 방법: 국민취업지원제도 1유형 대상자 경우 신청 > 수급자격 결정 및 알림 > 취업활동계획 수립 > 1차 구직촉진수당 지급 > 취업활동계획에 따른 구직활동 의무 이행 > 2~6회차 구직촉진수당 지급 > 사후관리  
🔗 신청하러 가기: [www.work24.go.kr](http://www.work24.go.kr)

---

### 2️⃣ 광주일자리종합지원센터 취업 지원
광주일자리종합지원센터 취업 지원은 구인·구직 맞춤형 취업지원사업을 통해 취업마인드 함양 및 취업률 제고에 도움이 되는 정책이야.

✨ 지원 내용: 청년·일자리로드쇼, 찾아가는 만남의 날, 찾아가는 일자리(job)카페 등 취업지원사업 추진  
📝 신청 자격: 19세 ~ 39세  
🗓️ 신청 기간: 상시 모집 또는 별도 공지  
🧾 신청 방법: 프로그램별 별도 공지에 따른 신청  
🔗 신청하러 가기: [https://www.gjjobgo.com/](https://www.gjjobgo.com/)

---

### 3️⃣ 자립·취업지원 서비스
자립·취업지원 서비스는 진로탐색과 취업 등에 관심이 있는 학교 밖 청소년이 자립 및 취업에 성공할 수 있도록 맞춤형 직업훈련, 일경험 및 취업연계 등을 지원하는 정책이야.

✨ 지원 내용: 진로상담, 직업역량강화 프로그램, 맞춤형 직업훈련 및 자격취득과정 지원, 직장체험 인턴십 등

In [16]:
# 질문-답변 테스트 2
question = '창업을 준비하는 청년을 위한 정책을 알려주세요'

print(f'👤 질문: {question}')
print('─' * 60)

result = rag.ask_with_sources(question)

print(f'\n🤖 답변:\n{result["answer"]}')
print(f'\n📚 참고 정책 ({result["retrieved_count"]}개):')
for i, source in enumerate(result['sources'], 1):
    print(f"  {i}. {source['정책명']} ({source['분야']})")

👤 질문: 창업을 준비하는 청년을 위한 정책을 알려주세요
────────────────────────────────────────────────────────────

🤖 답변:
오, 창업을 준비하는 청년을 위한 정책이 몇 개 있어! 도움이 될 만한 정보가 있어서 공유할게.

### 1️⃣ 청춘창고
청춘창고는 청년에게 창업 기회를 제공하고 창업 아이디어를 발굴할 수 있는 공간을 만들어주는 정책이야.

✨ 지원 내용: 청년 창업 인큐베이팅 및 창업 아이디어 발굴 공간 제공  
📝 신청 자격: 18세 ~ 45세 순천시 거주 청년  
🗓️ 신청 기간: 상시 신청 및 이메일(gusrud1419@korea.kr) 접수  
🧾 신청 방법: 상시 신청 및 이메일 접수  
🔗 신청하러 가기: N/A  

### 2️⃣ 청년 창업발굴 프로젝트
청년 창업발굴 프로젝트는 대학생에게 창업 아이디어를 발굴하고 창업 준비 기회를 제공하는 정책이야.

✨ 지원 내용: 창업 특강, 캠프, 경진대회, 시제품 제작 등 프로그램 운영  
📝 신청 자격: 제천시 관내 대학생 및 창업희망 청년(만 45세 이하)  
🗓️ 신청 기간: 20250301 ~ 20251231  
🧾 신청 방법: 세명대학교 취·창업지원처 홍보 및 접수 시 신청·참여  
🔗 신청하러 가기: N/A  

### 3️⃣ 청년창업농 디자인 제작 지원
이 정책은 청년 농업인에게 상품 판매에 필요한 디자인 제작과 홍보를 지원해주는 정책이야.

✨ 지원 내용: 디자인 제작·홍보 지원  
📝 신청 자격: 19세 ~ 39세 임대형 스마트팜 청년농업인  
🗓️ 신청 기간: N/A  
🧾 신청 방법: N/A  
🔗 신청하러 가기: N/A  

이건 진짜 놓치기 아쉬운 정책들이야. 조건 맞으면 꼭 한 번 확인해보는 거 추천해! 

더 궁금한 거 있으면 편하게 물어봐! 😊

📚 참고 정책 (5개):
  1. 2025 청년정책 발굴 사업 (정책인프라구축)
  2. 순천시 청춘창고 (창업)
  3. 청년창업농 디자인 제작 지원 (창업)
  4. 청년

In [17]:
# 질문-답변 테스트 3
question = '주거 지원 받을 수 있는 정책이 있을까요?'

print(f'👤 질문: {question}')
print('─' * 60)

result = rag.ask_with_sources(question)

print(f'\n🤖 답변:\n{result["answer"]}')
print(f'\n📚 참고 정책 ({result["retrieved_count"]}개):')
for i, source in enumerate(result['sources'], 1):
    print(f"  {i}. {source['정책명']} ({source['분야']})")

👤 질문: 주거 지원 받을 수 있는 정책이 있을까요?
────────────────────────────────────────────────────────────

🤖 답변:
오, 주거 지원에 대한 질문이구나! 요즘 주거 문제로 고민하는 친구들이 많아서 이런 정보는 정말 유용해. 

### 1️⃣ 주거취약계층 부동산 중개보수 지원
주거취약계층 부동산 중개보수 지원은 경제적으로 어려운 주거취약계층에게 주거 이전에 따른 주택 중개보수 비용 일부를 지원하여 경제적 부담을 덜어주는 정책이야. 

✨ 지원 내용: 주거취약계층 부동산 중개보수료 지원(30만원 이내)  
📝 신청 자격: 기초생활수급자, 차상위계층, 한부모가정, 중위소득 100% 이하 신혼부부(1회에 한하여 지원)  
🗓️ 신청 기간: 20250101 ~ 20251231  
🧾 신청 방법: 광주광역시 소재지 관할 자치구(부동산 담당 부서 및 동 행정복지센터)로 방문  
🔗 신청하러 가기: N/A  

### 2️⃣ 청소년자립지원관
청소년자립지원관은 가출 등 위기청소년을 조기에 발굴하고 보호하여 자립할 수 있도록 지원하는 정책이야. 

✨ 지원 내용: 시설(쉼터, 회복지원시설) 퇴소 후에도 자립기반 여건이 마련되지 않는 청소년에게 주거안정에 기반한 자립지원 서비스 제공  
📝 신청 자격: 19세 ~ 24세  
🗓️ 신청 기간: N/A  
🧾 신청 방법: N/A  
🔗 신청하러 가기: [청소년자립지원관](https://www.kyci.or.kr/userSite/Local_recovery/list.asp?basicNum=1)  

### 3️⃣ 신혼부부 주거지원 사업
신혼부부 주거지원 사업은 혼인신고일부터 장흥군 주소등록한 혼인기간 5년 이내 및 중위소득 200% 이하 신혼부부에게 주거비를 지원하는 정책이야. 

✨ 지원 내용: 
- 주택구입 및 전세 대출 이자지원: 기간 3년, 연대출 이자의 50% 이내 지원(연 최대 100만원) 
- 임대주택(월세) 주거비 지원: 기간 1년, 월 임대료 50% 이내(월 최대 

## 6. 대화형 챗봇

자유롭게 질문을 입력하여 테스트할 수 있습니다.

In [18]:
# 대화형 챗봇 함수
def interactive_chat(rag_system):
    """대화형 챗봇 실행"""
    print('\n' + '=' * 60)
    print('💬 청년 정책 RAG 챗봇')
    print('=' * 60)
    print('청년 정책에 대해 질문해주세요!')
    print('종료하려면 "quit", "q", "종료"를 입력하세요.\n')
    
    while True:
        try:
            question = input('\n👤 질문: ').strip()
            
            if question.lower() in ['quit', 'q', 'exit', '종료']:
                print('\n👋 챗봇을 종료합니다. 감사합니다!')
                break
            
            if not question:
                continue
            
            print('\n🔍 관련 정책을 검색하고 답변을 생성 중...')
            
            # 답변 생성
            result = rag_system.ask_with_sources(question)
            
            print(f'\n🤖 답변:\n{result["answer"]}')
            print(f'\n📚 참고 정책 ({result["retrieved_count"]}개):')
            for i, source in enumerate(result['sources'], 1):
                print(f"  {i}. {source['정책명']} ({source['분야']}) - {source['주관기관']}")
            
        except KeyboardInterrupt:
            print('\n\n👋 챗봇을 종료합니다.')
            break
        except Exception as e:
            print(f'\n❌ 오류 발생: {e}')

print('✅ 대화형 챗봇 함수 정의 완료')
print('아래 셀을 실행하면 대화형 모드가 시작됩니다.')

✅ 대화형 챗봇 함수 정의 완료
아래 셀을 실행하면 대화형 모드가 시작됩니다.


In [19]:
# 대화형 챗봇 실행 (필요시 주석 해제)
# interactive_chat(rag)

## 7. 직접 질문 테스트

아래 셀에서 직접 질문을 입력하여 테스트해보세요.

In [20]:
# 직접 질문 입력 (원하는 질문으로 수정하세요)
my_question = '취업 지원 정책 알려줘'

print(f'👤 질문: {my_question}')
print('─' * 60)

result = rag.ask_with_sources(my_question)

print(f'\n🤖 답변:\n{result["answer"]}')
print(f'\n📚 참고 정책 ({result["retrieved_count"]}개):')
for i, source in enumerate(result['sources'], 1):
    print(f"  {i}. {source['정책명']}")
    print(f"      분야: {source['분야']}")
    print(f"      신청기간: {source['신청기간']}")
    print(f"      주관기관: {source['주관기관']}")
    if source['신청URL'] != 'N/A' and source['신청URL']:
        print(f"      신청URL: {source['신청URL']}")
    print()

👤 질문: 취업 지원 정책 알려줘
────────────────────────────────────────────────────────────

🤖 답변:
오, 좋은 질문이야! 요즘 취업 지원 정책이 많이 나와서 도움이 될 거야. 몇 가지 정책을 소개해줄게!

### 1️⃣ 국민취업지원제도
국민취업지원제도는 청년실업자 등에게 원하는 일자리를 찾을 수 있도록 취업지원서비스와 소득지원을 함께 제공하여 노동시장 진입촉진과 경제적 자립기반 마련에 도움이 되는 정책이야.

✨ 지원 내용: 맞춤형 취업지원서비스와 소득지원 제공  
📝 신청 자격: 15세 ~ 34세  
🗓️ 신청 기간: 상시 모집 또는 별도 공지  
🧾 신청 방법: www.work24.go.kr  
🔗 신청하러 가기: [www.work24.go.kr](http://www.work24.go.kr)

---

### 2️⃣ 광주일자리종합지원센터 취업지원
광주일자리종합지원센터 취업지원은 취업정보 제공 및 취업지원 사업을 통해 지역 청년들의 취업 역량을 강화하고, 취업률을 높이고자 하는 정책이야.

✨ 지원 내용: 구인·구직 맞춤형 취업상담서비스 제공, 취업지원사업 실시  
📝 신청 자격: -  
🗓️ 신청 기간: 상시 모집 또는 별도 공지  
🧾 신청 방법: 광주경제진흥상생일자리재단 홈페이지(https://www.gjep.or.kr) 공지사항 등 확인, 광주일자리종합센터 직접 방문 및 유선상담 등  
🔗 신청하러 가기: N/A

---

### 3️⃣ 2024년 전통시장 청년상인 지원사업 안내
2024년 전통시장 청년상인 지원사업은 창업지원, 성장지원, 판로지원을 통해 청년 상인들을 돕는 정책이야.

✨ 지원 내용: 창업지원, 성장지원, 판로지원  
📝 신청 자격: 18세 ~ 39세  
🗓️ 신청 기간: 상시 모집 또는 별도 공지  
🧾 신청 방법: 지원별로 상이, 첨부된 파일 참고  
🔗 신청하러 가기: N/A

---

### 4️⃣ 광주일자리종합지원센터 취업 지원
광주일자리종합지원센터 취업 지원은 구인·구